Installation of the needed packages and import of the libraries.

In [ ]:
!pip install pyngrok
!pip install streamlit

import streamlit as st
from pyngrok import ngrok
import os
import subprocess
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


Connecting through the ngrok tunnel

In [ ]:
ngrok_key = open("ngrok.txt","r").read()

!ngrok config add-authtoken $ngrok_key

def run_streamlit():
  proc = subprocess.Popen(["streamlit", "run", "Culinary_Compass_Tool.py", "--server.port=8501"])
  time.sleep(5)
  return proc

def start_ngrok():
  ngrok.kill()
  url = ngrok.connect(8501, "http")
  print("Streamlit URL:", url)
  return url

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Setting up the Streamlit app.

In [ ]:
open("Culinary_Compass_Tool.py","w").write("""
import streamlit as st
import pandas as pd
import numpy as np
import os
import sys
import logging
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

st.image('dishes.png')
st.title(':blue[Culinary Compass Dishes Recommendation]')
st.header(':blue[Select the cuisine and flavor of preference]', divider = True)

st.header("First dataset")
menu_item_data = pd.read_excel('Weighted_Menu_Items.xlsx')
menu_item_data['menu_item'] = menu_item_data['menu_item'].fillna('')
cuisine_list = menu_item_data['cuisine'].sort_values().unique()
cuisine_dropdown = st.selectbox('Select your favorite cuisine', cuisine_list, index = 0)
flavor_list = menu_item_data['dominant_flavor'].sort_values().unique()
flavor_dropdown = st.selectbox('Select your favorite flavor', flavor_list, index = 0)
choice = st.button('Show me recommendations')
container = st.container()
st.divider()

st.header("Second dataset")
menu_items = pd.read_excel('Menu Items_Synthetic Dataset.xlsx')
menu_items['menu_item'] = menu_items['menu_item'].fillna('')
cuisines_list = menu_items['cuisine'].sort_values().unique()
cuisines_dropdown = st.selectbox('Select your favorite cuisine', cuisines_list, index = 0)
flavors_list = menu_items['dominant_flavor'].sort_values().unique()
flavors_dropdown = st.selectbox('Select your favorite flavor', flavors_list, index = 0, key = 'Flavors')
choice1 = st.button('Show me recommendations', key = 'Cuisines/Flavors')
containers = st.container()
st.divider()

st.header("Third dataset")
celine_menu_items = pd.read_excel('Celine_weighted_menu_items.xlsx')
celine_menu_items['menu_item'] = celine_menu_items['menu_item'].fillna('')
celine_list = celine_menu_items['cuisine'].sort_values().unique()
celine_dropdown = st.selectbox('Select your favorite cuisine', celine_list, index = 0)
celine_flavor_list = celine_menu_items['dominant_flavor'].sort_values().unique()
celine_flavor_dropdown = st.selectbox('Select your favorite flavor', celine_flavor_list, index = 0, key = "Celine / flavor")
choice2 = st.button('Show me recommendations', key = 'Celine')
celine_container = st.container()
st.divider()

def load_menu_data(file_path: str) -> pd.DataFrame:
  if not os.path.exists(file_path):
      logging.error(f"File not found: {file_path}")
      raise FileNotFoundError(f"The file {file_path} does not exist.")
  logging.info(f"Loading file: {file_path}")
  return pd.read_excel(file_path)

def process_data(df: pd.DataFrame) -> np.hstack:
  df['flavour_weight'] = df['flavour_weight'].apply(eval)
  flavor_df = pd.json_normalize(df['flavour_weight'])
  scaler = StandardScaler()
  scaled_flavors = scaler.fit_transform(flavor_df)
  tfidf = TfidfVectorizer(stop_words='english')
  ingredient_tfidf = tfidf.fit_transform(df['ingredients'])
  combined_features = np.hstack([scaled_flavors, ingredient_tfidf.toarray()])
  return combined_features


def recommend_cosine_sim(menu, cuisine, taste_profile, combined_features, top_n=5):
  taste_profile = taste_profile.strip().lower()
  taste_profile_dic = {taste_profile: 1.0}
  valid_flavors = {"sweet", "salty", "sour", "bitter", "umami"}
  flavor_profile = {flavor: 0.0 for flavor in valid_flavors}
  for key in taste_profile_dic:
        if key in flavor_profile:
            flavor_profile[key] = taste_profile_dic[key]
        else:
            raise ValueError(f"Invalid flavor: {key}. Must be one of {list(valid_flavors)}")
  filtered_df = menu[menu['cuisine'].str.strip().str.lower() == cuisine.strip().lower()].copy()
  filtered_df = filtered_df[filtered_df['dominant_flavor'].str.strip().str.lower() == taste_profile].copy()
  if filtered_df.empty:
    return f"No menu items found for cuisine: {cuisine}"
  flavor_profiles = np.array([
        list(flavor.values()) if isinstance(flavor, dict) else [0] * len(flavor_profile)
        for flavor in filtered_df['flavour_weight']])
  input_profile = np.array(list(flavor_profile.values())).reshape(1, -1)
  cosine_sim = cosine_similarity(input_profile, flavor_profiles).flatten()
  cosine_dist = 1 - cosine_sim
  sim_scores = np.argsort(cosine_dist)[:top_n]
  recommendations = filtered_df.iloc[sim_scores][['menu_item_id', 'menu_item', 'cuisine', 'ingredients', 'dominant_flavor']]
  recommendations['score'] = cosine_dist[sim_scores]
  return recommendations

def get_restaurant_name(name):
  name = name[:-3]
  name = name.replace("_", " ")
  name = name.title()
  return name

def main(path, cuisine_input, flavor_input, container):
  try:
        menu = load_menu_data(path) #Loads excel into df

        cuisine = cuisine_input
        taste_profile = flavor_input
        combined_features = process_data(menu)
        n = 5
        visualize = False
        if (type(recommend_cosine_sim(menu, cuisine, taste_profile, combined_features, n)) == str):
          container.write("No menu items found for your choice of cuisine (" + cuisine_input + ") and flavor (" + flavor_input + ").")
        else:
          container.write("Based on your choice of cuisine (" + cuisine_input + ") and flavor (" + flavor_input + "), we recommend the following menu items:")
          res_cos = pd.DataFrame(recommend_cosine_sim(menu, cuisine, taste_profile, combined_features, n))
          num_rows = res_cos.shape[0]
          for i in range(num_rows):
            container.write("Choice " + str(i+1) + ": " + res_cos['menu_item'].iloc[i] + " at " + get_restaurant_name(res_cos['menu_item_id'].iloc[i]))
          container.subheader("The full dataframe")
          container.table(res_cos[['cuisine','menu_item','ingredients','dominant_flavor','score']])

  except Exception as e:
      logging.error(f"An error occurred: {e}")
      st.write(logging.error(f"An error occurred: {e}"))
      raise

#code to keep the data in session
if "choice_pressed" not in st.session_state:
  st.session_state.choice_pressed = False
if "choice1_pressed" not in st.session_state:
  st.session_state.choice1_pressed = False
if "choice2_pressed" not in st.session_state:
  st.session_state.choice2_pressed = False


if(choice):
  main('Weighted_Menu_Items.xlsx', cuisine_dropdown, flavor_dropdown, container)
  st.session_state.choice_pressed = True

if(choice1):
  main('Menu Items_Synthetic Dataset.xlsx', cuisines_dropdown, flavors_dropdown, containers)
  st.session_state.choice1_pressed = True


if(choice2):
  main('Celine_weighted_menu_items.xlsx', celine_dropdown, celine_flavor_dropdown, celine_container)
  st.session_state.choice2_pressed = True
""")

7781

Launching the Streamlit App.

In [ ]:
streamlit_proc = run_streamlit()
public_url = start_ngrok()

Streamlit URL: NgrokTunnel: "https://b44c-34-169-105-99.ngrok-free.app" -> "http://localhost:8501"
